#📌 Extracão

In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)

# Visualizar os primeiros dados
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformação

In [ ]:
# Expandir as colunas que contêm dicionários
df_expanded = pd.concat([
    df.drop(['customer', 'phone', 'internet', 'account'], axis=1),
    df['customer'].apply(pd.Series),
    df['phone'].apply(pd.Series),
    df['internet'].apply(pd.Series),
    df['account'].apply(pd.Series)
], axis=1)

# Visualizar os primeiros dados achatados
df_expanded.head()

In [ ]:
# Primeiro, checar se o valor é um dicionário antes de expandir
def expand_charges(val):
    if isinstance(val, dict):
        return pd.Series(val)
    return pd.Series({'Monthly': None, 'Total': None})

charges_df = df_expanded['Charges'].apply(expand_charges)
charges_df.columns = ['MonthlyCharges', 'TotalCharges']

# Concatenar com o dataframe original
df_expanded = pd.concat([df_expanded.drop('Charges', axis=1), charges_df], axis=1)

In [ ]:
df_expanded.head()

In [ ]:
df_expanded.isnull().sum()
# Verifica quantos valores ausentes (NaN) existem em cada coluna.

In [ ]:
df_expanded.duplicated().sum()

In [ ]:
df_expanded.info()  # Exibe o resumo das colunas com tipos e contagem de valores não nulos.
df_expanded.describe(include='all')  # Mostra estatísticas descritivas para colunas numéricas e categóricas.

In [ ]:
# Criando a nova coluna 'Contas_Diarias' com base no faturamento mensal dividido por 30
df_expanded['Contas_Diarias'] = df_expanded['MonthlyCharges'] / 30  # Calcula o valor médio diário

# Visualiza as primeiras linhas para confirmar
df_expanded[['MonthlyCharges', 'Contas_Diarias']].head()

In [ ]:
# Converte valores de 'Yes' e 'No' em colunas categóricas para 1 e 0
colunas_binarias = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

for coluna in colunas_binarias:
    # Aplicar a transformação no DataFrame correto (df_expanded)
    df_expanded[coluna] = df_expanded[coluna].astype(str).str.strip().str.lower().map({'yes': 1, 'no': 0})

# Visualizar as primeiras linhas para confirmar as mudanças
df_expanded[colunas_binarias].head()

In [ ]:
# Renomeia colunas para nomes mais claros (exemplo de algumas)
# Aplique as renomeações em df_expanded
df_expanded.rename(columns={
    'MonthlyCharges': 'Faturamento_Mensal',
    'TotalCharges': 'Faturamento_Total',
    'Churn': 'Cancelou',
    'gender': 'gênero'
}, inplace=True)

# Remove espaços extras e coloca letras minúsculas em colunas de texto
# Selecione colunas do tipo 'object' do df_expanded
colunas_categoricas = df_expanded.select_dtypes(include='object').columns

for col in colunas_categoricas:
    # Verifique se a coluna realmente contém dados que podem ser tratados como string antes de aplicar .str
    if df_expanded[col].apply(lambda x: isinstance(x, str)).any():
        df_expanded[col] = df_expanded[col].str.strip().str.lower()
    else:
        print(f"Warning: Skipping non-string column '{col}' for string operations.")

In [ ]:
df_expanded.head()

#📊 Carga e análise

In [ ]:
# Exibe estatísticas como média, mediana, desvio padrão, mínimo, máximo e quartis
df_expanded.describe().T  # .T exibe as estatísticas na vertical para facilitar a leitura

In [ ]:
# Filtra apenas colunas numéricas
numericas = df_expanded.select_dtypes(include='number')

# Calcula estatísticas personalizadas
print("Média:\n", numericas.mean())
print("\nMediana:\n", numericas.median())
print("\nDesvio padrão:\n", numericas.std())
print("\nValores mínimos:\n", numericas.min())
print("\nValores máximos:\n", numericas.max())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define o estilo do gráfico
sns.set(style="whitegrid")

# Gráfico de barras com contagem de clientes que cancelaram e os que não cancelaram
sns.countplot(x='Cancelou', data=df_expanded, palette='pastel')

plt.title('Distribuição de Cancelamento de Clientes')
plt.xlabel('Cancelou (0 = Não, 1 = Sim)')
plt.ylabel('Quantidade de Clientes')
plt.xticks([0, 1], ['Não Cancelou', 'Cancelou'])  # Rótulos personalizados
plt.show()

In [ ]:
# Contagem de cancelamentos
valores = df_expanded['Cancelou'].value_counts()
labels = ['Não Cancelou', 'Cancelou']
cores = ['#8fd9b6', '#ff9999']

# Gráfico de pizza
plt.pie(valores, labels=labels, autopct='%1.1f%%', startangle=90, colors=cores, wedgeprops={'edgecolor': 'black'})
plt.title('Proporção de Clientes que Cancelaram')
plt.axis('equal')  # Garante formato circular
plt.show()

In [ ]:
# Define o estilo dos gráficos
sns.set(style="whitegrid")

# Lista de colunas categóricas para comparar com a evasão
variaveis_categoricas = [
    'gênero', 'Contract', 'PaymentMethod', 'PhoneService',
    'MultipleLines', 'OnlineSecurity', 'TechSupport', 'Partner'
]

# Print the columns of df_expanded to check for 'phoneservice'
print("Columns in df_expanded:", df_expanded.columns)

# Cria um gráfico de barras para cada variável categórica
for var in variaveis_categoricas:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=var, hue='Cancelou', data=df_expanded, palette='pastel')
    plt.title(f'Cancelamento por {var.capitalize()}')
    plt.xlabel(var.capitalize())
    plt.ylabel('Quantidade de Clientes')
    plt.legend(title='Cancelou', labels=['Não', 'Sim'])
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.show()

In [ ]:

# Lista de variáveis numéricas para analisar
variaveis_numericas = ['Faturamento_Total', 'Faturamento_Mensal', 'Contas_Diarias', 'tenure']

# Cria histogramas comparando quem cancelou e quem não
for var in variaveis_numericas:
    plt.figure(figsize=(6, 4))
    sns.histplot(data=df_expanded, x=var, hue='Cancelou', bins=30, kde=True, palette='pastel', element='step')
    plt.title(f'Distribuição de {var} por Cancelamento')
    plt.xlabel(var)
    plt.ylabel('Número de Clientes')
    plt.legend(title='Cancelou', labels=['Não', 'Sim'])
    plt.tight_layout()
    plt.show()

In [ ]:

# Seleciona apenas variáveis numéricas
numericas = df_expanded.select_dtypes(include=['int64', 'float64'])

# Calcula correlação
correlacao = numericas.corr()

# Visualiza a matriz
plt.figure(figsize=(12, 8))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('📊 Matriz de Correlação entre Variáveis Numéricas')
plt.show()

#📄Relatorio Final

# 🔍 Análise de Evasão de Clientes (Churn)

## 📌 Introdução

O objetivo deste projeto é analisar dados de clientes de uma empresa de telecomunicações para entender o comportamento de evasão (churn). O churn representa os clientes que encerraram seus contratos, e entender os fatores que levam a isso é essencial para desenvolver estratégias de retenção e fidelização.

---

## 🧹 Limpeza e Tratamento de Dados

Durante esta etapa, foram realizadas as seguintes ações:

- **Importação dos dados brutos** de um arquivo `.csv`.
- **Transformação de colunas aninhadas (JSON)** em colunas regulares.
- **Conversão de variáveis categóricas** para formatos numéricos (por exemplo, "Sim"/"Não" para 1/0), o que facilita a análise.
- **Remoção de valores ausentes e duplicados**, garantindo a integridade dos dados.
- **Criação da coluna `Contas_Diarias`**, que representa o valor médio pago por dia, derivado do faturamento mensal dividido por 30 dias.
- **Renomeação de colunas** para termos mais compreensíveis, facilitando a leitura e interpretação dos resultados.

---

## 📊 Análise Exploratória de Dados (EDA)

A análise exploratória permite observar o comportamento dos dados e identificar padrões associados à evasão de clientes.

### 🎯 Distribuição Geral do Cancelamento

Visualizamos a proporção de clientes que cancelaram vs. os que permaneceram utilizando gráficos de barras e de pizza. Isso ajuda a entender a gravidade do churn no negócio.

### 👥 Cancelamento por Variáveis Categóricas

Foram analisadas variáveis como:

- Gênero (`gender`)
- Tipo de contrato (`contract`)
- Método de pagamento (`paymentmethod`)
- Serviços adicionais como suporte técnico e segurança online
- Presença de dependente ou parceiro

Com essas comparações, foi possível observar **quais grupos de clientes estão mais propensos a cancelar**.

### 📈 Cancelamento por Variáveis Numéricas

Variáveis como:

- `Faturamento_Total`
- `Faturamento_Mensal`
- `tenure` (tempo de contrato)
- `Contas_Diarias`

foram analisadas por meio de **boxplots** e **histogramas** para verificar diferenças estatísticas no comportamento entre clientes que cancelaram e os que não cancelaram.

---

## 🧠 Conclusões e Insights

A partir da análise, os principais achados foram:

- **Clientes com contratos mensais** têm muito mais chance de cancelamento do que os com contratos anuais.
- **Clientes sem suporte técnico ou segurança online** apresentaram taxas mais altas de evasão.
- A maioria dos cancelamentos ocorre **nos primeiros meses do contrato**, indicando a importância da retenção inicial.
- **Clientes com menor faturamento total** e **valores diários mais altos** mostraram maior propensão ao churn.

---

## 💡 Recomendações

Com base nas descobertas, recomenda-se:

1. **Oferecer benefícios ou descontos em contratos mais longos** (anual ou bianual).
2. **Incluir serviços adicionais como suporte técnico gratuito** nos primeiros meses.
3. **Focar em ações de retenção nos 3 primeiros meses de contrato.**
4. **Identificar clientes com alto custo diário** e atuar preventivamente com atendimento proativo.
5. **Incentivar métodos de pagamento automáticos** para evitar cancelamentos por esquecimento ou inadimplência.

---

## ✅ Encerramento

A análise permitiu mapear os principais fatores que influenciam a evasão de clientes. Com essas informações, a empresa pode desenvolver estratégias mais eficazes de retenção, melhorando a experiência do cliente e reduzindo perdas financeiras.